#TRAFFIC LIGHT & COLOR DETECTION



---




##Setup installation


---



###TensorFlow version = 1.X

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__) 

###Restart RUNTIME after selected version of tensorflow in case of failure or Restart

In [0]:
import os
os.kill(os.getpid(), 9)

###Mount drive (Every Time)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###Clone the Tensorflow models repository and the SmartScooter repo from github (Just 1 Time)

Change directory to SMART_SCOOTER

In [0]:
%cd '/content/drive/My Drive/SMART_SCOOTER/'

In [0]:
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/Eugenill/SmartScooter.git
!cp -r pycocotools /content/models/research/

###Install needed libraries


In [0]:
%cd '/content/drive/My Drive/SMART_SCOOTER/'
#1. Install some needed tools
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
#2. Compile the model definitionn
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/
!protoc object_detection/protos/*.proto --python_out=.
#3. Set the environment
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/SMART_SCOOTER/models/research/:/content/drive/My Drive/SMART_SCOOTER/models/research/slim'
#4. Always run this every restart of session
!python setup.py build
!python setup.py install
#5. Test to see if all we need for the training has been installed
!python object_detection/builders/model_builder_test.py

##Prepare images and annotations


---







###Import images 

Save the images in `SMART_SCOOTER/images/folder_name`

###Import annotations

####Pascal format (.xml) - Other traffic light images

Save the image xml annotations in `SMART_SCOOTER/train_xml_annotations or test_xml_annotations `

Remember to specify the path to the image in **filename**, but just from the SMART_SCOOTER/images/ folder.

EXAMPLE
```
<annotation>
	<folder></folder>
	<filename>rgb_jpg/train/.....</filename>
	<path></path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>300</width>
		<height>300</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>cone</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>300</xmin>
			<ymin>200</ymin>
			<xmax>310</xmax>
			<ymax>210</ymax>
		</bndbox>
	</object>
</annotation>
```



We will copy the xml annotations to `SMART_SCOOTER/annotations/TL/train_xml_annotations/` and create the `test_xml_annotations` folder too

In [0]:
%cd '/content/drive/My Drive/SMART_SCOOTER/annotations/TL'
%cp new_train_xml train_xml_annotations
%mkdir test_xml_annotations

#### Bosch format (.yaml)

To use this annotations we first have to convert the train and test.yaml to xml files that we are going to save in the same folder as the the ones we already have saved.
In: `train and test_xml_annotations`



We have the yaml's in `SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS/label_files`



```
#-------------NEW
	onlypath=impath.split('./')[1]
	second_folder=onlypath.split('/').[1]
	imagename = impath.split('/')[-1]
	path_w_noimage=onlypath.split(imagename)[0]
	currentfolder = savedir.split("\\")[-1]
	if 'train' in onlypath:		
		if second_folder not in os.listdir("/content/drive/My Drive/SMART_SCOOTER/images/rgb_jpg/train"): 
			os.mkdir("/content/drive/My Drive/SMART_SCOOTER/images/rgb_jpg/train/"+second_folder)
	if imagename in os.listdir("/content/drive/My Drive/SMART_SCOOTER/images/"+path_w_noimage):
#--------------

```

- In one hand we have to create those folders of images we dont have in our drive, but we have their annotation (just to not get errors).

- Secondly we are just going to create those xml of the images we have in our image folders.







INPUTS: 
1. yaml files

OUTPUT:
1. Xml annotations saved in the folder specified

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS
!python bosch_to_pascal.py 'label_files/train.yaml' '/content/drive/My Drive/SMART_SCOOTER/annotations/TL/train_xml_annotations/'
!python bosch_to_pascal.py 'label_files/test.yaml' '/content/drive/My Drive/SMART_SCOOTER/annotations/TL/test_xml_annotations/'

###Create csv files

Change paths to search the xml and to save the csv files in `xml_to_csv.py` : 

```
def main():
  for folder in ['train', 'test']:
        image_path = os.path.join(os.getcwd(), ('/content/drive/My Drive/SMART_SCOOTER/annotations/TL/'+folder+'_xml_annotations'))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv(('/content/drive/My Drive/SMART_SCOOTER/annotations/TL/'+folder+'_csv_annotations/'+folder+'_labels.csv'), index=None)
  print('Successfully converted xml to csv.')

```
  



This script was originally from: https://github.com/datitran/raccoon_dataset

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS
!python xml_to_csv.py

##Create train and test.record

---



To create the record files we have to use `generate_tfrecord.py` already downloaded and modified.  


The .records are files that collects all those images labeled (empty or not) in an 'array' with: `filename (and its folder), height, weigth, class, and position of the object box`. FOR EVERY IMAGE LABELED ONLY.

The .record's are the only files that contains information of the dataset for the training.


Modifications:

```
def class_text_to_int(row_label):
    if row_label == 'Off':
        return 1
    elif row_label == 'Green':
        return 2
    elif row_label == 'Yellow':
        return 3
    elif row_label == 'Red':
        return 4
    else:
        return 0
```


This script was originally from: https://github.com/datitran/raccoon_dataset

INPUTS: 
1. CSV files
2. Image principal folder

OUTPUT:
1. record file, saved in ../record/

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS
!python generate_tfrecord.py --csv_input='/content/drive/My Drive/SMART_SCOOTER/annotations/TL/train_csv_annotations/train_labels.csv' --image_dir='/content/drive/My Drive/SMART_SCOOTER/images' --output_path='/content/drive/My Drive/SMART_SCOOTER/record_files/train.record'
!python generate_tfrecord.py --csv_input='/content/drive/My Drive/SMART_SCOOTER/annotations/TL/test_csv_annotations/test_labels.csv' --image_dir='/content/drive/My Drive/SMART_SCOOTER/images' --output_path='/content/drive/My Drive/SMART_SCOOTER/record_files/test.record'

##Create labelmap, ckpt and config files

---



###Labelmap.pbtxt

This file tells the training which labels (classes) we want as output in the model. We are going to "attach" this file in the config

We find this file in: ```SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS/labelmap```


```
item {
  id: 1
  name: 'off'
}

item {
  id: 2
  name: 'Green'
}

item {
  id: 3
  name: 'Yellow'
}

item {
  id: 4
  name: 'Red'
}
```



###model.ckpt

We can download any pre-trained Object detection model from tensorflow with the code below. In this case we are downloading the ssdlite_mobilenet_v2

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!tar -xvf ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz 

From this downloaded model we will use the pretrained model: 
```model.ckpt```
This model has two parts:
1. Mobilenet for feature extraction model
2. SSDlite_v2 for finetuning

In the training process we will import all the pretrained mobilenet part, and we will modify the SSD part (finetunning).


###Config

The config file tells the training how we are going to train the model and with which files.

In this case the config file is also originally from the ssdlite_mobilenet_v2 model. But with the next modifications:



1. `num_classes=4`

2. `set anchor boxes aspect ratios`

3. `fine_tune_checkpoint: "/content/drive/My Drive/SMART_SCOOTER/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt"`

  or

  `fine_tune_checkpoint: "/content/drive/My Drive/SMART_SCOOTER/new_models/TL_XX/model.ckpt-XXXX"`

3. `set batch_size=24`

5. `num_steps=10000-15000`

4. `input_path: "/content/drive/My Drive/SMART_SCOOTER/record_files/train.record"`

5. `input_path: "/content/drive/My Drive/SMART_SCOOTER/record_files/test.record"`

6. `label_map_path: "/content/drive/My Drive/SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS/labelmap/TL.pbtxt"`

7. ```
image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
```
8. `num_examples: 1438 (test images)`




We will find the config in: `SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS/config`

##Training

---



Open your google drive tab and go to Legacy folder in the object detection directory, copy or move the train.py file into the object detection folder. then go back to Colab and run the training with the code below. train_dir (train.py), model_dir (model_main.py)

In [0]:
%tensorboard --logdir /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/
!python model_main.py --logtostderr --model_dir='/content/drive/My Drive/SMART_SCOOTER/training/TL_V2/' --pipeline_config_path='/content/drive/My Drive/SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS/config/TL.config'

##Export your inference graph

---



In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection
!python export_inference_graph.py --input_type image_tensor  --pipeline_config_path '/content/drive/My Drive/SMART_SCOOTER/SmartScooter/TRAFFIC_LIGHTS/TL_W_COLORS/config/TL.config'   --trained_checkpoint_prefix training/TL_V1/model.ckpt-15000   --output_directory new_models/TL_V1

Zip your inference saved on object_detection

In [0]:
!zip -r TL_V1.zip new_models/TL_V1

Create files to run the network on opencv. 
In other words, we need to create a graph.pbtxtt, but we already have it from the training itself

In [0]:
!wget https://github.com/opencv/opencv/blob/master/samples/dnn/tf_text_graph_ssd.py
!wget https://github.com/opencv/opencv/blob/master/samples/dnn/tf_text_graph_common.py

In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/
!python tf_text_graph_ssd.py --input new_models/TL_V1/frozen_inference_graph.pb --config training/TL_V1/TL_V1.config --output new_models/TL_V1/graph.pbtxt

/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection
Scale: [0.200000-0.950000]
Aspect ratios: [1.0, 2.0, 0.5, 3.0, 0.3333]
Reduce boxes in the lowest layer: True
Number of classes: 4
Number of layers: 6
box predictor: convolutional
Input image size: 300x300


##Test your inference graph

---




In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection


/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection


In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/utils
# This is needed since the notebook is stored in the object_detection folder.

sys.path.append("..")
import utils
#sys.path.append("..")
import label_map_util

import visualization_utils as vis_util

/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/utils


In [0]:
%cd /content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed to display the images.
%matplotlib inline


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/new_models/TL_V1/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/training/TL_V1', 'TL_V1.pbtxt')

NUM_CLASSES = 4

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')



label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/images/rgb_jpg/train/2015-10-05-14-40-46_bag'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}'.format(i)) for i in os.listdir('/content/drive/My Drive/SMART_SCOOTER/models/research/object_detection/images/rgb_jpg/train/2015-10-05-14-40-46_bag') ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
c=0
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
         if c<6: 
          if image_path.endswith('.jpg'):
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            c+=1

Output hidden; open in https://colab.research.google.com to view.